<a href="https://colab.research.google.com/github/jimmymanianchira/FastAi_NLP/blob/master/Transfer_learning_nlp_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files


Quick Start: Training an IMDb sentiment model with ULMFiT

In [0]:
from fastai.text import *

In [3]:
URLs.IMDB_SAMPLE

'http://files.fast.ai/data/examples/imdb_sample'

In [0]:
path = untar_data(URLs.IMDB_SAMPLE)

In [5]:

df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [0]:

data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=8)

In [0]:

data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [0]:

bs=192
data_lm = load_data(path, 'data_lm_export.pkl', bs=bs)
data_clas = load_data(path, 'data_clas_export.pkl', bs=8)

In [11]:
torch.cuda.current_device()

0

In [12]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [13]:

learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.381495,3.946625,0.282366,00:18


In [14]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4,1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,4.075387,3.828389,0.294851,00:23
1,3.867562,3.783039,0.296726,00:23
2,3.671761,3.775161,0.298735,00:23


In [15]:
learn.predict("This is a review about", n_words=10)

'This is a review about called reds in Work . An 1933 study'

In [16]:
learn.predict("I really love ", n_words=10)

"I really love  Gene 's wife , but i found it hard"

In [0]:
learn.save('ft')
learn.save_encoder('ft_enc')

In [7]:

learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5).to_fp16()
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (799 items)
x: TextList
xxbos xxmaj evidently when you offer a actor enough money they will do anything . i am not sure how much xxmaj john xxmaj xxunk - xxmaj xxunk got , but most of the money he made should go to his fans as an apology for even being associated with such a xxup rotten movie . xxmaj the special effects were worse then effects from the 1950 's b movies and the acting of the rest of the cast was even worse . xxmaj as to how bad the acting was a child gave the second best performance in my opinion . xxmaj the xxmaj english was terribly xxunk and i think no one could really even speak xxmaj english they just xxunk how the words should sound instead of xxunk the script and trying to make their character both " life - like " and real .,xxbos xxmaj terry xxmaj gilliam 's stunning feature - length adaptation of xxmaj chris xxmaj marker 's short film xxup la xxup xxunk is full of mind - xxunk surprises , yet still touches yo

In [19]:
data_clas.show_batch()

text,target
"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n \n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , steaming bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj",negative
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the sweetest and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj this film sat on my xxmaj tivo for weeks before i watched it . i dreaded a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the screwed - up xxunk of xxmaj new xxmaj yorkers . \n \n xxmaj the format is the same as xxmaj max xxmaj xxunk ' "" xxmaj la xxmaj ronde",positive
"xxbos xxmaj many neglect that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first stealth games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - rounded gaming experience in general . xxmaj with graphics",positive
"xxbos i really wanted to love this show . i truly , honestly did . \n \n xxmaj for the first time , gay viewers get their own version of the "" xxmaj the xxmaj bachelor "" . xxmaj with the help of his obligatory "" hag "" xxmaj xxunk , xxmaj james , a good looking , well - to - do thirty - something has the chance",negative


In [8]:

learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.564125,0.533226,0.731343,00:46


In [9]:

learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4, 1e-2))


epoch,train_loss,valid_loss,accuracy,time
0,0.511699,0.502037,0.741294,01:32
1,0.442990,0.391530,0.810945,01:30
2,0.322487,0.381992,0.825871,01:28


Note to Myself-

Due to memory constraints in Colab, I ran the code in 2 blocks. The part upto the language model training, I trained and saved and I restarted the kernel. From there I started again. Also I have reduced the batch size to use for the text classification learner.